**Import necessary libraries**
- sqlalchemy/create_engine in order to write our sql queries
- pandas for data manipulation
- FileLink to output results into excel workbooks
- re in order to utilize regex, which is needed to replace a few unwanted characters in the data
- openpyxl for modifying excel data

In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from IPython.display import FileLink
import re
import openpyxl

__Prepare the data for analysis__

 Before we start diving into the data we need to load it from the csv file and take a few quick steps to clean it up.

- Remove unwanted characters from the original csv that get in the way of insuring the columns fit into the necessary data types
- Use sqlalchemy create_engine to create a sqlite database to load our data into
- Convert the data (df) to sql
- Run a quick test to insure sql queries are working properly (This particular index had values that were causing issues in it)

In [2]:
df = pd.read_csv("C:/Users/bkpug/Downloads/FastFoodNutritionMenuV3.csv")
df.columns = df.columns.str.strip().str.replace(r'\n|\(.*?\)', '', regex=True).str.replace(' ', '_')
engine = create_engine('sqlite:///:memory:')
df.to_sql('data', con=engine, index=False, if_exists='replace')

with engine.begin() as conn:
    result = conn.execute(text("SELECT * FROM data")).fetchall()

result[475]

('Burger King', '29 fl oz', 410.0, 0.0, 0.0, '0', 0.0, '0', '135', '111', '0', '109', 0.0, 519.0)

__Write a quick query to check the data types for the sql db__

In [3]:
with engine.begin() as conn:
    result = conn.execute(text("PRAGMA table_info(data)")).fetchall()
    for row in result:
        print(row)

(0, 'Company', 'TEXT', 0, None, 0)
(1, 'Item', 'TEXT', 0, None, 0)
(2, 'Calories', 'FLOAT', 0, None, 0)
(3, 'Calories_fromFat', 'FLOAT', 0, None, 0)
(4, 'Total_Fat', 'FLOAT', 0, None, 0)
(5, 'Saturated_Fat', 'TEXT', 0, None, 0)
(6, 'Trans_Fat', 'FLOAT', 0, None, 0)
(7, 'Cholesterol', 'TEXT', 0, None, 0)
(8, 'Sodium_', 'TEXT', 0, None, 0)
(9, 'Carbs', 'TEXT', 0, None, 0)
(10, 'Fiber', 'TEXT', 0, None, 0)
(11, 'Sugars', 'TEXT', 0, None, 0)
(12, 'Protein', 'FLOAT', 0, None, 0)
(13, 'Weight_WatchersPnts', 'FLOAT', 0, None, 0)


__After checking the data types I noticed the 'Sugars' column wasn't any form of numerical data, which I need it to be for a certain query.__

The below code should not only fix that but also replace any instance of '<1' in the data

In [4]:
df['Sugars'].replace('<1', '0', inplace=True)

df['Sugars'] = df['Sugars'].astype(float)

df['Sugars'].value_counts()

Sugars
0.0      205
2.0       96
1.0       61
3.0       56
4.0       42
        ... 
118.0      1
85.0       1
99.0       1
138.0      1
112.0      1
Name: count, Length: 120, dtype: int64

In [5]:
df.to_sql('data', con=engine, index=False, if_exists='replace')

1134

__Now we begin our analysis__

___The goal of this project is to identify the best options for a few specific diets in the fast food industry___

There are 6 restaurants available in this dataset.
- McDonalds
- Burger King
- Wendys
- KFC
- Taco Bell
- Pizza Hut

For each of these restaurants I would like to find the best possible food options for protein focused, calorie counting focused, Low sugar, 
and red meat free diets. For each diet I would like there to be multiple options accounting for the dietary focus. 
On the first two I woule like to assign a minimum of 10g of protein, and a maximum of 300 calories.
This assures that any 2 items bought from this list will maintain a caloric intake of 600 or less while provided plenty of protein as well.
The second two queries will be more directly focused on their own dietary restrictions and less on calories vs protein.

__After each set of queries I will be adding each of the results into an excel worksheet which will be downloadable simply by clicking on it.__


In [6]:
with engine.begin() as conn:
    mcdonalds_by_protein = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'McDonald’s' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10")).fetchall()

mcdonalds_by_protein

[('McDonald’s', 'Premium Bacon Ranch Salad with Grilled Chicken', 260.0, 33.0),
 ('McDonald’s', 'Premium Caesar Salad with Grilled Chicken', 220.0, 30.0),
 ('McDonald’s', 'Ranch Snack Wrap® (Grilled)', 270.0, 18.0),
 ('McDonald’s', 'Honey Mustard Snack Wrap® (Grilled)', 260.0, 18.0),
 ('McDonald’s', 'Chipotle BBQ Snack Wrap® (Grilled)', 260.0, 18.0),
 ('McDonald’s', 'Egg McMuffin®', 300.0, 18.0),
 ('McDonald’s', 'Cheeseburger', 300.0, 15.0),
 ('McDonald’s', 'Chicken McNuggets® (6 piece)', 280.0, 13.0),
 ('McDonald’s', 'Hamburger', 250.0, 12.0),
 ('McDonald’s', 'Sausage Burrito', 300.0, 12.0)]

In [7]:
with engine.begin() as conn:
    mcdonalds_by_calories = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'McDonald’s' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10")).fetchall()

mcdonalds_by_calories

[('McDonald’s', 'Cheeseburger', 300.0, 15.0),
 ('McDonald’s', 'Egg McMuffin®', 300.0, 18.0),
 ('McDonald’s', 'Sausage Burrito', 300.0, 12.0),
 ('McDonald’s', 'Iced Nonfat Caramel Mocha (Large)', 300.0, 11.0),
 ('McDonald’s', 'Chicken McNuggets® (6 piece)', 280.0, 13.0),
 ('McDonald’s', 'Nonfat Caramel Mocha (Large)', 280.0, 12.0),
 ('McDonald’s', 'Ranch Snack Wrap® (Grilled)', 270.0, 18.0),
 ('McDonald’s', 'Honey Mustard Snack Wrap® (Grilled)', 260.0, 18.0),
 ('McDonald’s', 'Chipotle BBQ Snack Wrap® (Grilled)', 260.0, 18.0),
 ('McDonald’s', 'Premium Bacon Ranch Salad with Grilled Chicken', 260.0, 33.0)]

__Side note:__
The results for the low sugar query output a large list that is a bit disruptive, so while this code cell looks different than others it is for simplification.

In [59]:
with engine.begin() as conn:
    mcdonalds_low_sugar = conn.execute(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'McDonald’s' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC")).fetchall()

from IPython.display import display, HTML

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in mcdonalds_low_sugar:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [9]:
with engine.begin() as conn:
    mcdonalds_no_red = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'McDonald’s' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC")).fetchall()

mcdonalds_no_red

[('McDonald’s', 'Chicken McNuggets® (4 piece)', 190.0, 9.0),
 ('McDonald’s', 'Premium Caesar Salad with Grilled Chicken', 220.0, 30.0),
 ('McDonald’s', 'Chicken McNuggets® (6 piece)', 280.0, 13.0),
 ('McDonald’s', 'Premium Southwest Salad with Grilled Chicken', 320.0, 30.0),
 ('McDonald’s', 'Premium Caesar Salad with Crispy Chicken', 330.0, 26.0),
 ('McDonald’s', 'McChicken ®', 360.0, 14.0),
 ('McDonald’s', 'Premium Grilled Chicken Classic Sandwich', 360.0, 27.0),
 ('McDonald’s', '4 Piece Chicken McNuggets Happy Meal', 395.0, 19.0),
 ('McDonald’s', 'Southern Style Crispy Chicken Sandwich', 400.0, 24.0),
 ('McDonald’s', 'Chicken Selects® Premium Breast Strips (3 pc)', 400.0, 23.0),
 ('McDonald’s', 'Southern Style Chicken Biscuit (Regular Size Biscuit)', 410.0, 17.0),
 ('McDonald’s', 'Premium Southwest Salad with Crispy Chicken', 430.0, 26.0),
 ('McDonald’s', 'Premium Grilled Chicken Club Sandwich', 460.0, 35.0),
 ('McDonald’s', 'Chicken McNuggets® (10 piece)', 470.0, 22.0),
 ('McDonald’

In [10]:
# This will be for making the excel sheet for all of mcdonalds options

with engine.begin() as conn:
    mcdonalds_by_protein1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'McDonald’s' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10"), conn)
    mcdonalds_by_calories1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'McDonald’s' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10"), conn)
    mcdonalds_low_sugar1 = pd.read_sql(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'McDonald’s' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC"), conn)
    mcdonalds_no_red1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'McDonald’s' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC"), conn)

with pd.ExcelWriter("mcdonalds_nutrition.xlsx") as writer:
    mcdonalds_by_protein1.to_excel(writer, sheet_name="Protein Focused Diet", index=False)
    mcdonalds_by_calories1.to_excel(writer, sheet_name="Calorie Focused Diet", index=False)
    mcdonalds_low_sugar1.to_excel(writer, sheet_name="Low Sugar Diet", index=False)
    mcdonalds_no_red1.to_excel(writer, sheet_name="No Red Meat", index=False)

FileLink("mcdonalds_nutrition.xlsx")

C:\Users\bkpug\PycharmProjects\AutoIT2\mcdonalds_nutrition.xlsx

In [11]:
with engine.begin() as conn:
    bk_by_protein = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Burger King' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10")).fetchall()

bk_by_protein

[('Burger King', 'Cheeseburger', 280.0, 15.0),
 ('Burger King', 'Cheeseburger', 280.0, 15.0),
 ('Burger King', 'Hamburger', 240.0, 13.0),
 ('Burger King', 'Chicken Fries – 9 pc.', 280.0, 13.0),
 ('Burger King', 'Hamburger', 240.0, 13.0),
 ('Burger King', 'Chicken Nuggets- 6pc', 260.0, 12.0),
 ('Burger King', 'Chicken Nuggets- 6pc', 260.0, 12.0),
 ('Burger King', 'PB&J Jamwich', 300.0, 11.0)]

In [12]:
with engine.begin() as conn:
    bk_by_calories = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Burger King' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10")).fetchall()

bk_by_calories

[('Burger King', 'PB&J Jamwich', 300.0, 11.0),
 ('Burger King', 'Cheeseburger', 280.0, 15.0),
 ('Burger King', 'Chicken Fries – 9 pc.', 280.0, 13.0),
 ('Burger King', 'Cheeseburger', 280.0, 15.0),
 ('Burger King', 'Chicken Nuggets- 6pc', 260.0, 12.0),
 ('Burger King', 'Chicken Nuggets- 6pc', 260.0, 12.0),
 ('Burger King', 'Hamburger', 240.0, 13.0),
 ('Burger King', 'Hamburger', 240.0, 13.0)]

In [58]:
with engine.begin() as conn:
    bk_low_sugar = conn.execute(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'Burger King' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC")).fetchall()

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in bk_low_sugar:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [14]:
with engine.begin() as conn:
    bk_no_red = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Burger King' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC")).fetchall()

bk_no_red

[('Burger King', 'Chicken Nuggets- 4pc', 170.0, 8.0),
 ('Burger King', 'Chicken Nuggets- 4pc', 170.0, 8.0),
 ('Burger King', 'Spicy Chicken Nuggets- 4pc', 210.0, 8.0),
 ('Burger King', 'Chicken Nuggets- 6pc', 260.0, 12.0),
 ('Burger King', 'Chicken Nuggets- 6pc', 260.0, 12.0),
 ('Burger King', 'Chicken Fries – 9 pc.', 280.0, 13.0),
 ('Burger King', 'Spicy Chicken Nuggets- 6pc', 320.0, 12.0),
 ('Burger King', 'Spicy Crispy Chicken Jr.', 390.0, 12.0),
 ('Burger King', 'Chicken Nuggets- 10pc', 430.0, 20.0),
 ('Burger King', 'Garden Chicken Salad with Crispy Chicken – no dressing', 440.0, 25.0),
 ('Burger King', 'Crispy Chicken Jr.', 450.0, 12.0),
 ('Burger King', 'Spicy Chicken Nuggets- 10 pc', 530.0, 20.0),
 ('Burger King', 'Club Salad with Crispy Chicken – no dressing', 540.0, 31.0)]

In [15]:
with engine.begin() as conn:
    bk_by_protein1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Burger King' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10"), conn)
    bk_by_calories1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Burger King' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10"), conn)
    bk_low_sugar1 = pd.read_sql(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'Burger King' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC"), conn)
    bk_no_red1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Burger King' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC"), conn)

with pd.ExcelWriter("burger_king_nutrition.xlsx") as writer:
    bk_by_protein1.to_excel(writer, sheet_name="Protein Focused Diet", index=False)
    bk_by_calories1.to_excel(writer, sheet_name="Calorie Focused Diet", index=False)
    bk_low_sugar1.to_excel(writer, sheet_name="Low Sugar Diet", index=False)
    bk_no_red1.to_excel(writer, sheet_name="No Red Meat", index=False)

FileLink("burger_king_nutrition.xlsx")

C:\Users\bkpug\PycharmProjects\AutoIT2\burger_king_nutrition.xlsx

In [16]:
with engine.begin() as conn:
    wendys_by_protein = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Wendy’s' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10")).fetchall()

wendys_by_protein

[('Wendy’s', 'Berry Burst Chicken Salad, Half Size', 290.0, 23.0),
 ('Wendy’s', 'Chili, Large', 250.0, 23.0),
 ('Wendy’s', 'Southwest Avocado Chicken Salad, Half Size', 300.0, 21.0),
 ('Wendy’s', 'Grilled Chicken Wrap', 270.0, 20.0),
 ('Wendy’s', 'Kids Grilled Chicken Wrap', 270.0, 20.0),
 ('Wendy’s', 'Jr. Cheeseburger', 280.0, 16.0),
 ('Wendy’s', 'Chili, Small', 170.0, 15.0),
 ('Wendy’s', 'Kids 2 Piece Chicken Tenders', 200.0, 15.0),
 ('Wendy’s', 'Kids Cheeseburger', 280.0, 15.0),
 ('Wendy’s', '6 Picece Chicken Nuggets', 250.0, 13.0)]

In [17]:
with engine.begin() as conn:
    wendys_by_calories = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Wendy’s' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10")).fetchall()

wendys_by_calories

[('Wendy’s', 'Southwest Avocado Chicken Salad, Half Size', 300.0, 21.0),
 ('Wendy’s', 'Berry Burst Chicken Salad, Half Size', 290.0, 23.0),
 ('Wendy’s', 'Jr. Cheeseburger', 280.0, 16.0),
 ('Wendy’s', 'Kids Cheeseburger', 280.0, 15.0),
 ('Wendy’s', 'Grilled Chicken Wrap', 270.0, 20.0),
 ('Wendy’s', 'Kids Grilled Chicken Wrap', 270.0, 20.0),
 ('Wendy’s', '6 Picece Chicken Nuggets', 250.0, 13.0),
 ('Wendy’s', 'Chili, Large', 250.0, 23.0),
 ('Wendy’s', 'Kids Hamburger', 240.0, 13.0),
 ('Wendy’s', 'Kids 2 Piece Chicken Tenders', 200.0, 15.0)]

In [57]:
with engine.begin() as conn:
    wendys_low_sugar = conn.execute(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'Wendy’s' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC")).fetchall()

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in wendys_low_sugar:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [60]:
with engine.begin() as conn:
    wendys_no_red = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Wendy’s' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC")).fetchall()

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in wendys_no_red:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [20]:
with engine.begin() as conn:
    wendys_by_protein1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Wendy’s' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10"), conn)
    wendys_by_calories1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Wendy’s' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10"), conn)
    wendys_low_sugar1 = pd.read_sql(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'Wendy’s' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC"), conn)
    wendys_no_red1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Wendy’s' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC"), conn)

with pd.ExcelWriter("wendys_nutrition.xlsx") as writer:
    wendys_by_protein1.to_excel(writer, sheet_name="Protein Focused Diet", index=False)
    wendys_by_calories1.to_excel(writer, sheet_name="Calorie Focused Diet", index=False)
    wendys_low_sugar1.to_excel(writer, sheet_name="Low Sugar Diet", index=False)
    wendys_no_red1.to_excel(writer, sheet_name="No Red Meat", index=False)

FileLink("wendys_nutrition.xlsx")

C:\Users\bkpug\PycharmProjects\AutoIT2\wendys_nutrition.xlsx

In [21]:
with engine.begin() as conn:
    kfc_by_protein = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'KFC' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10")).fetchall()

kfc_by_protein

[('KFC', 'Breast', 260.0, 38.0),
 ('KFC', 'KENTUCKY GRILLED CHICKEN Breast', 210.0, 38.0),
 ('KFC', 'KFC® Gizzards', 300.0, 20.0),
 ('KFC', 'Limited Time ORIGINAL RECIPE CHICKEN Chicken Thigh', 280.0, 19.0),
 ('KFC', 'EXTRA CRISPY  Tender (each)', 260.0, 19.0),
 ('KFC', 'Thigh', 180.0, 17.0),
 ('KFC', 'KENTUCKY GRILLED CHICKEN Thigh', 150.0, 17.0),
 ('KFC', 'KFC® Livers', 230.0, 17.0),
 ('KFC', 'Chicken Littles', 300.0, 14.0),
 ('KFC', 'SPICY CRISPY CHICKEN Thigh', 270.0, 13.0)]

In [22]:
with engine.begin() as conn:
    kfc_by_calories = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'KFC' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10")).fetchall()

kfc_by_calories

[('KFC', 'Chicken Littles', 300.0, 14.0),
 ('KFC', 'KFC® Gizzards', 300.0, 20.0),
 ('KFC', 'Chicken Whole Wing', 290.0, 10.0),
 ('KFC', 'POPCORN NUGGETS Kids', 290.0, 13.0),
 ('KFC', 'Limited Time ORIGINAL RECIPE CHICKEN Chicken Thigh', 280.0, 19.0),
 ('KFC', 'Sweet Kernel Corn (Family)', 280.0, 10.0),
 ('KFC', 'SPICY CRISPY CHICKEN Thigh', 270.0, 13.0),
 ('KFC', 'KFC® Famous Bowl – Snack Size', 270.0, 11.0),
 ('KFC', 'Breast', 260.0, 38.0),
 ('KFC', 'EXTRA CRISPY  Tender (each)', 260.0, 19.0)]

In [61]:
with engine.begin() as conn:
    kfc_low_sugar = conn.execute(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'KFC' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC")).fetchall()

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in kfc_low_sugar:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [62]:
with engine.begin() as conn:
    kfc_no_red = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'KFC' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC")).fetchall()

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in kfc_no_red:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [25]:
with engine.begin() as conn:
    kfc_by_protein1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'KFC' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10"), conn)
    kfc_by_calories1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'KFC' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10"), conn)
    kfc_low_sugar1 = pd.read_sql(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'KFC' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC"), conn)
    kfc_no_red1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'KFC' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC"), conn)

with pd.ExcelWriter("kfc_nutrition.xlsx") as writer:
    kfc_by_protein1.to_excel(writer, sheet_name="Protein Focused Diet", index=False)
    kfc_by_calories1.to_excel(writer, sheet_name="Calorie Focused Diet", index=False)
    kfc_low_sugar1.to_excel(writer, sheet_name="Low Sugar Diet", index=False)
    kfc_no_red1.to_excel(writer, sheet_name="No Red Meat", index=False)

FileLink("kfc_nutrition.xlsx")

C:\Users\bkpug\PycharmProjects\AutoIT2\kfc_nutrition.xlsx

In [26]:
with engine.begin() as conn:
    taco_bell_by_protein = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Taco Bell' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10")).fetchall()

taco_bell_by_protein

[('Taco Bell', 'Soft Taco – Chicken', 180.0, 13.0),
 ('Taco Bell', 'Soft Taco – Chicken', 160.0, 12.0),
 ('Taco Bell', 'Chicken Chipotle Melt – Value Menu', 190.0, 12.0),
 ('Taco Bell', 'Loaded Nacho Taco', 250.0, 10.0),
 ('Taco Bell', 'Loaded Nacho Taco (New)', 250.0, 10.0),
 ('Taco Bell', 'Soft Taco – Beef', 210.0, 10.0),
 ('Taco Bell', 'Soft Taco Supreme – Specialties', 210.0, 10.0),
 ('Taco Bell', 'Loaded Nacho Taco – Value Menu', 250.0, 10.0)]

In [27]:
with engine.begin() as conn:
    taco_bell_by_calories = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Taco Bell' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10")).fetchall()

taco_bell_by_calories

[('Taco Bell', 'Loaded Nacho Taco', 250.0, 10.0),
 ('Taco Bell', 'Loaded Nacho Taco (New)', 250.0, 10.0),
 ('Taco Bell', 'Loaded Nacho Taco – Value Menu', 250.0, 10.0),
 ('Taco Bell', 'Soft Taco – Beef', 210.0, 10.0),
 ('Taco Bell', 'Soft Taco Supreme – Specialties', 210.0, 10.0),
 ('Taco Bell', 'Chicken Chipotle Melt – Value Menu', 190.0, 12.0),
 ('Taco Bell', 'Soft Taco – Chicken', 180.0, 13.0),
 ('Taco Bell', 'Soft Taco – Chicken', 160.0, 12.0)]

In [63]:
with engine.begin() as conn:
    taco_bell_low_sugar = conn.execute(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'Taco Bell' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC")).fetchall()

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in taco_bell_low_sugar:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [29]:
with engine.begin() as conn:
    taco_bell_no_red = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Taco Bell' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC")).fetchall()

taco_bell_no_red

[('Taco Bell', 'Soft Taco – Chicken', 160.0, 12.0),
 ('Taco Bell', 'Soft Taco – Chicken', 180.0, 13.0),
 ('Taco Bell', 'Chicken Chipotle Melt – Value Menu', 190.0, 12.0),
 ('Taco Bell', 'Chalupa Supreme® – Chicken', 330.0, 16.0),
 ('Taco Bell', 'Burrito Supreme® – Chicken', 370.0, 19.0),
 ('Taco Bell', 'Power Menu Bowl – Chicken', 470.0, 26.0),
 ('Taco Bell', 'Quesadilla – Chicken', 510.0, 27.0)]

In [30]:
with engine.begin() as conn:
    taco_bell_by_protein1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Taco Bell' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10"), conn)
    taco_bell_by_calories1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Taco Bell' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10"), conn)
    taco_bell_low_sugar1 = pd.read_sql(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'Taco Bell' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC"), conn)
    taco_bell_no_red1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Taco Bell' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC"), conn)

with pd.ExcelWriter("taco_bell_nutrition.xlsx") as writer:
    taco_bell_by_protein1.to_excel(writer, sheet_name="Protein Focused Diet", index=False)
    taco_bell_by_calories1.to_excel(writer, sheet_name="Calorie Focused Diet", index=False)
    taco_bell_low_sugar1.to_excel(writer, sheet_name="Low Sugar Diet", index=False)
    taco_bell_no_red1.to_excel(writer, sheet_name="No Red Meat", index=False)

FileLink("taco_bell_nutrition.xlsx")

C:\Users\bkpug\PycharmProjects\AutoIT2\taco_bell_nutrition.xlsx

In [31]:
with engine.begin() as conn:
    pizza_hut_by_protein = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Pizza Hut' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10")).fetchall()

pizza_hut_by_protein

[('Pizza Hut', 'Hand Tossed Slices Hawaiian Chicken Large', 280.0, 15.0),
 ('Pizza Hut', 'Hand Tossed Slices Backyard BBQ Chicken Large', 300.0, 14.0),
 ('Pizza Hut', 'Detroit Double Cheesy Pizza Slice', 280.0, 13.0),
 ('Pizza Hut', 'Detroit Supremo Pizza Slice', 290.0, 13.0),
 ('Pizza Hut', 'Pan Pizza Slices Pepperoni Lover’s® Medium', 300.0, 13.0),
 ('Pizza Hut', 'Hand Tossed Slices  Buffalo Chicken Large', 270.0, 13.0),
 ('Pizza Hut', 'Hand Tossed Slices Cheese Large', 290.0, 13.0),
 ('Pizza Hut', 'Hand Tossed Slices Pepperoni Large', 300.0, 13.0),
 ('Pizza Hut', 'Pan Pizza Slices Chicken Bacon Parmesan Medium', 270.0, 12.0),
 ('Pizza Hut', 'Pan Pizza Slices Hawaiian Chicken Medium', 240.0, 12.0)]

In [32]:
with engine.begin() as conn:
    pizza_hut_by_calories = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Pizza Hut' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10")).fetchall()

pizza_hut_by_calories

[('Pizza Hut', 'Pan Pizza Slices Pepperoni Lover’s® Medium', 300.0, 13.0),
 ('Pizza Hut', 'Hand Tossed Slices Backyard BBQ Chicken Large', 300.0, 14.0),
 ('Pizza Hut', 'Hand Tossed Slices Pepperoni Large', 300.0, 13.0),
 ('Pizza Hut', 'Detroit Supremo Pizza Slice', 290.0, 13.0),
 ('Pizza Hut', 'Hand Tossed Slices Cheese Large', 290.0, 13.0),
 ('Pizza Hut', 'Hand Tossed Slices Meat Lover’s® Medium', 290.0, 12.0),
 ('Pizza Hut', 'Detroit Double Cheesy Pizza Slice', 280.0, 13.0),
 ('Pizza Hut', 'Hand Tossed Slices Beyond Meat® Pepperoni Large', 280.0, 12.0),
 ('Pizza Hut', 'Hand Tossed Slices Hawaiian Chicken Large', 280.0, 15.0),
 ('Pizza Hut', 'Pan Pizza Slices Backyard BBQ Chicken Medium', 270.0, 11.0)]

In [64]:
with engine.begin() as conn:
    pizza_hut_low_sugar = conn.execute(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'Pizza Hut' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC")).fetchall()

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in pizza_hut_low_sugar:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [65]:
with engine.begin() as conn:
    pizza_hut_no_red = conn.execute(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Pizza Hut' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC")).fetchall()

html = "<div style='height:200px; overflow:auto; border:1px solid #ccc; padding:10px;'>"
for row in pizza_hut_no_red:
    html += f"{row}<br>"
html += "</div>"

display(HTML(html))

In [35]:
with engine.begin() as conn:
    pizza_hut_by_protein1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Pizza Hut' AND Calories <= 300 AND Protein >= 10 ORDER BY Protein DESC LIMIT 10"), conn)
    pizza_hut_by_calories1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Pizza Hut' AND Calories <= 300 AND Protein >= 10 ORDER BY Calories DESC LIMIT 10"), conn)
    pizza_hut_low_sugar1 = pd.read_sql(text("SELECT Company, Item, Calories, Sugars FROM data WHERE Company = 'Pizza Hut' AND Sugars <= 10 AND Calories <= 300 ORDER BY Sugars ASC"), conn)
    pizza_hut_no_red1 = pd.read_sql(text("SELECT Company, Item, Calories, Protein FROM data WHERE Company = 'Pizza Hut' AND Calories <= 600 AND Item LIKE '%Chicken%' AND Item NOT LIKE '%without chicken%' AND Item NOT LIKE '%Bacon%' AND Item NOT LIKE '%BLT%' ORDER BY Calories ASC"), conn)

with pd.ExcelWriter("pizza_hut_nutrition.xlsx") as writer:
    pizza_hut_by_protein1.to_excel(writer, sheet_name="Protein Focused Diet", index=False)
    pizza_hut_by_calories1.to_excel(writer, sheet_name="Calorie Focused Diet", index=False)
    pizza_hut_low_sugar1.to_excel(writer, sheet_name="Low Sugar Diet", index=False)
    pizza_hut_no_red1.to_excel(writer, sheet_name="No Red Meat", index=False)

FileLink("pizza_hut_nutrition.xlsx")

C:\Users\bkpug\PycharmProjects\AutoIT2\pizza_hut_nutrition.xlsx